In [77]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import shap

## START HERE IF YOU HAVE THE ABOVE FILE ALREADY

In [78]:
df = pd.read_csv('/Users/yenhann/Documents/MIT/15.572 Analytics Lab/Datasets/feed_into_pipeline_data.csv')

## Adding Summaries and People **Rankings**

In [79]:
summaries = pd.read_csv('/Users/yenhann/Documents/MIT/15.572 Analytics Lab/Datasets/GlobalOriginalsWithSummary.csv')[['combined_title', 'summary']]
people_rankings = pd.read_csv('/Users/yenhann/Documents/MIT/15.572 Analytics Lab/Datasets/showsWithPrincipalRankings.csv')[['combined_title', 'director_rank', 'writer_rank', 'actor_rank']]

In [80]:
df = df.merge(summaries, on='combined_title')
df = df.merge(people_rankings, on='combined_title')

In [81]:
df.drop_duplicates(inplace=True)

## Split the data into movies and tv, then train/test split

In [82]:
#Split movie and tv_show
movies, tv_shows = df[df['is_tv_show']==False], df[df['is_tv_show']==True]

In [83]:
# remove tv show rows with season >= 24
# THERE ARE 2 TV SHOWS WITH SEASON = 24 AND SEASON = 2806. NEED TO REMOVE
tv_shows = tv_shows[tv_shows['season']<24]

# drop combined_title, release_date, lower_title, is_tv_show, tconst
tv_shows = tv_shows.drop(['combined_title', 'release_date', 'lower_title', 'is_tv_show', 'tconst'], axis=1)
tv_shows

,release_year,is_top10,season,averageRating,numVotes,titleType,runtimeMinutes,genres,release_date_quarter,release_date_month,release_date_week,summary,director_rank,writer_rank,actor_rank
0,2020,0,1.0,6.8,4915,tvSeries,36.0,Comedy,2,4,16,A father takes an irreverent and honest approa...,360.0,3596.000000,14244.333330
1,2020,0,1.0,6.4,1099,tvSeries,NaN,Documentary,3,8,33,This docu-series takes a deep dive into the lu...,16655.5,NaN,27266.000000
3,2020,0,1.0,5.8,4179,tvSeries,NaN,Reality-TV,1,3,11,A social experiment where 100 individuals will...,3227.0,22674.000000,34331.666670
6,2020,1,4.0,7.5,291371,tvSeries,60.0,"Drama,Mystery,Thriller",2,6,23,"Follows teenager Clay Jensen, in his quest to ...",50.0,1193.666667,1760.333333
11,2020,1,4.0,7.3,25863,tvSeries,49.0,"Action,Drama,Sci-Fi",3,8,33,A post-apocalyptic thriller set in near-future...,7572.0,18213.333330,15519.666670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2215,2021,1,1.0,5.7,1486,tvSeries,35.0,"Comedy,Drama,Family",1,3,11,Talented teen figure skater Kayla is forced to...,7234.0,23511.333330,141848.666700
2217,2021,0,1.0,5.7,1344,tvSeries,27.0,"Action,Comedy,Drama",2,4,16,A teen with the power of invisibility is dragg...,16260.5,37014.166670,134824.500000
2219,2022,1,1.0,8.0,9215,tvMiniSeries,90.0,"Documentary,Music",1,2,7,"Follows the life of Kanye West, in an intimate...",17577.5,NaN,20629.166670
2220,2020,0,1.0,2.6,3361,tvSeries,30.0,"Documentary,Reality-TV",1,1,4,"Leading with curiosity, Gwyneth Paltrow and he...",NaN,NaN,28078.500000


In [84]:
# train test split with stratified sampling
from sklearn.model_selection import train_test_split

tvshows_X_train, tvshows_X_test, tvshows_y_train, tvshows_y_test = train_test_split(tv_shows.drop('is_top10', axis=1), tv_shows['is_top10'], test_size=0.2, random_state=42, stratify=tv_shows['is_top10'])

## Hugging Face PCA

In [85]:
# import summary_transformer_embeddings.csv
summary_embeddings = pd.read_csv('/Users/yenhann/Documents/MIT/15.572 Analytics Lab/Datasets/summary_transformer_embeddings.csv')
summary_embeddings_df = summary_embeddings.copy()

In [86]:
summary_embeddings_df

,summary_embedding_0,summary_embedding_1,summary_embedding_2,summary_embedding_3,summary_embedding_4,summary_embedding_5,summary_embedding_6,summary_embedding_7,summary_embedding_8,summary_embedding_9,...,summary_embedding_374,summary_embedding_375,summary_embedding_376,summary_embedding_377,summary_embedding_378,summary_embedding_379,summary_embedding_380,summary_embedding_381,summary_embedding_382,summary_embedding_383
0,-0.026633,0.018920,-0.055134,0.045714,-0.063023,-0.032902,-0.024073,-0.049662,0.021096,-0.002936,...,0.073990,-0.024292,-0.000128,0.056980,-0.071850,0.032407,0.101245,0.041416,0.107043,-0.010314
1,0.008881,-0.022368,0.007538,0.018812,-0.003038,0.020658,-0.104257,-0.010471,-0.032509,-0.071558,...,-0.009092,0.048180,0.053298,-0.018979,-0.034616,-0.023176,0.066810,-0.137848,0.092657,0.052799
2,-0.023224,-0.003328,-0.020494,0.026740,-0.009808,-0.004658,-0.028434,-0.025144,0.017680,-0.029676,...,0.073531,-0.027467,0.035513,-0.008700,0.048015,0.016851,0.014063,-0.039804,-0.037779,-0.008573
3,-0.019948,0.036421,-0.032200,0.017349,-0.001934,-0.001572,0.068681,-0.014159,-0.022527,0.040237,...,0.090168,0.066829,0.020300,0.105946,-0.012541,0.031277,0.135142,0.099034,-0.108421,-0.074679
4,0.007733,0.069801,-0.040708,0.076647,-0.006824,0.048367,0.059733,-0.014537,-0.030768,-0.005307,...,-0.013327,-0.027364,-0.082106,0.015057,-0.101740,-0.069111,0.029374,-0.031163,0.035734,0.029763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2221,0.065544,-0.016539,0.012974,-0.012867,0.042465,-0.021782,0.080996,-0.081871,0.007024,-0.049119,...,0.045124,-0.071766,-0.008403,0.036693,0.010890,0.043983,0.011136,-0.017340,0.033225,-0.048873
2222,0.016400,-0.043311,0.054263,0.004811,-0.032548,0.047001,0.047832,-0.020683,-0.009917,-0.056862,...,-0.027186,0.055188,-0.066201,0.048929,-0.124170,-0.050052,0.110961,-0.032043,-0.007126,0.028938
2223,0.065066,-0.001605,0.029295,-0.036900,-0.028460,0.093836,0.075292,-0.038419,-0.066347,-0.012721,...,0.028930,-0.037840,-0.103201,0.142967,-0.044767,0.014078,0.010336,0.031978,0.020591,-0.086957
2224,-0.018001,-0.053426,-0.103662,0.075346,-0.018186,0.045000,0.067063,-0.010451,0.007042,0.041649,...,-0.015691,-0.069962,0.021161,0.045569,-0.052063,0.110053,-0.054747,-0.060268,0.044384,-0.028002


In [87]:
# apply PCA to summary_embeddings_df
from sklearn.decomposition import PCA

nComponents = 200
pca = PCA(n_components=nComponents)
summary_embeddings_pca_df = pca.fit_transform(summary_embeddings_df)

In [88]:
pd.DataFrame(summary_embeddings_pca_df)

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.031450,0.247994,-0.030031,-0.204703,0.070657,0.078839,-0.149140,-0.244862,-0.009455,-0.054442,...,0.027827,-0.039109,0.034277,0.000786,0.086170,0.015878,-0.017109,0.002477,-0.022311,0.027452
1,-0.227742,-0.098685,-0.137212,0.138005,0.122655,-0.020513,0.098578,-0.082827,0.061267,-0.067376,...,0.036195,-0.011996,0.049776,0.001679,-0.007247,-0.019362,0.037846,-0.034694,0.075539,-0.049901
2,-0.117267,0.045462,0.121812,-0.121147,-0.024666,0.059354,0.095709,0.119830,-0.033386,-0.101316,...,-0.015913,0.014628,-0.038425,-0.022794,0.055197,0.009645,0.019875,-0.051361,-0.040597,0.021217
3,-0.080910,-0.044301,0.098781,0.299778,0.112112,-0.042162,0.060013,-0.137956,0.055825,0.181168,...,-0.025781,-0.049271,0.004663,-0.040111,0.014523,0.036474,0.059100,0.000810,-0.021229,-0.061341
4,-0.042086,0.031152,0.069962,-0.224520,0.303595,-0.039167,-0.174553,-0.107022,-0.025669,-0.095999,...,-0.004830,-0.044130,-0.005496,0.015346,0.004990,-0.010267,0.061171,0.018912,-0.017143,0.023983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2221,-0.168729,-0.019834,-0.310371,-0.123630,-0.041336,-0.102093,-0.089942,-0.231081,-0.159561,0.016990,...,-0.000916,-0.013949,0.028340,0.026787,-0.036238,-0.002959,-0.063136,0.033502,-0.052925,-0.015980
2222,-0.212929,-0.019776,-0.023201,0.198410,-0.000943,-0.136511,-0.024948,-0.036193,0.180170,0.058768,...,-0.020307,-0.002931,-0.008551,-0.044015,0.043990,-0.035414,0.005142,-0.013141,0.055445,0.057520
2223,-0.199548,0.103003,-0.122135,-0.103774,-0.051027,-0.113499,-0.104250,-0.320650,-0.134817,-0.003452,...,0.006840,0.044596,-0.011639,-0.035004,-0.052217,-0.066339,-0.025348,0.023518,-0.027101,-0.062005
2224,0.254890,0.105878,-0.266983,0.275503,-0.063390,-0.139316,-0.020193,-0.011852,-0.036879,-0.051363,...,0.002867,0.003938,-0.028282,-0.028035,0.023660,0.003411,-0.010648,0.019226,-0.015646,0.021361


In [89]:
print("The total explained variance by", nComponents, "components = ", 100*sum(pca.explained_variance_),"%")

The total explained variance by 200 components =  80.71420282923171 %


## XGBoost - No Summary

In [90]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [91]:
rankings_pipeline = Pipeline([
    #('imputer', SimpleImputer(strategy='mean')),
    ('logger', FunctionTransformer(np.log1p))
])

xgb_transformer = ColumnTransformer(
    [
        ("onehot_titleType",OneHotEncoder(handle_unknown='ignore'), ['titleType']),
        ("onehot_release_year",OneHotEncoder(handle_unknown='ignore'), ['release_year']),
        #("bagofwords", CountVectorizer(stop_words='english', min_df=0.02), 'summary'),             # One hot encoding
        #("tfidf", TfidfVectorizer(stop_words='english', min_df=0.02), 'summary'),                  # TF-IDF
        ("genres_count", CountVectorizer(min_df=0.05), 'genres'),
        #("sentence_transformer", embedder, 'summary'),                                             # hugging face - PCA
        #("imputer", SimpleImputer(strategy='mean'), ['averageRating', 'numVotes', 'runtimeMinutes']),
        ("rankings_pipeline", rankings_pipeline, ['director_rank', 'actor_rank', 'writer_rank']),
    ],
    remainder='passthrough')

In [92]:
# Recreate pipeline using XGBoost
xgb_pipe = Pipeline(
    steps=[
        ('xgb_transformer', xgb_transformer),
        #('normalizer', sklearn.preprocessing.Normalizer()),
        ('xgb', XGBClassifier())
    ]
)

xgb_gs = GridSearchCV(
    xgb_pipe,
    param_grid={
        'xgb__max_depth': [3, 5, 7],
        'xgb__learning_rate': [0.01, 0.1, 0.5],
        'xgb__n_estimators': [100, 200, 300]
    },
    scoring={"AUC": "roc_auc", "Accuracy": "accuracy"},
    refit="AUC",
    cv=5,
    verbose=0,
    n_jobs=-1,
)

In [93]:
# fit xgb_gs
xgb_gs.fit(tvshows_X_train.drop(columns=['summary']), tvshows_y_train)

/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future ve

[16:20:29] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:20:29] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:20:29] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to '

/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost

[16:20:30] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:20:30] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:20:30] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to '

/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost

...

/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:21:34] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:21:34] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[16:21:37] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('xgb_transformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('onehot_titleType',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['titleType']),
                                                                        ('onehot_release_year',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['release_year']),
                                                                        ('genres_count',
                                                                         CountVectorizer(min_df=0.05),
                                                   

In [94]:
print("XGBoost on TV SHOWS (NO SUMMARY) best score =", xgb_gs.best_score_)

XGBoost on TV SHOWS (NO SUMMARY) best score = 0.8210334170250905


In [95]:
# get auc of test set
from sklearn.metrics import roc_auc_score
print("Test set AUC (NO SUMMARY) = ", roc_auc_score(tvshows_y_test, xgb_gs.predict_proba(tvshows_X_test)[:,1]))

Test set AUC (NO SUMMARY) =  0.8626173462239036


## XGBoost - Bag of Words Summary

In [96]:
rankings_pipeline = Pipeline([
    #('imputer', SimpleImputer(strategy='mean')),
    ('logger', FunctionTransformer(np.log1p))
])

xgb_transformer = ColumnTransformer(
    [
        ("onehot_titleType",OneHotEncoder(handle_unknown='ignore'), ['titleType']),
        ("onehot_release_year",OneHotEncoder(handle_unknown='ignore'), ['release_year']),
        ("bagofwords", CountVectorizer(stop_words='english', min_df=0.02), 'summary'),             # One hot encoding
        #("tfidf", TfidfVectorizer(stop_words='english', min_df=0.02), 'summary'),                  # TF-IDF
        ("genres_count", CountVectorizer(min_df=0.05), 'genres'),
        #("sentence_transformer", embedder, 'summary'),                                             # hugging face - PCA
        #("imputer", SimpleImputer(strategy='mean'), ['averageRating', 'numVotes', 'runtimeMinutes']),
        ("rankings_pipeline", rankings_pipeline, ['director_rank', 'actor_rank', 'writer_rank']),
    ],
    remainder='passthrough')


In [97]:
# Recreate pipeline using XGBoost
xgb_pipe = Pipeline(
    steps=[
        ('xgb_transformer', xgb_transformer),
        #('normalizer', sklearn.preprocessing.Normalizer()),
        ('xgb', XGBClassifier())
    ]
)

xgb_gs = GridSearchCV(
    xgb_pipe,
    param_grid={
        'xgb__max_depth': [3, 5, 7],
        'xgb__learning_rate': [0.01, 0.1, 0.5],
        'xgb__n_estimators': [100, 200, 300]
    },
    scoring={"AUC": "roc_auc", "Accuracy": "accuracy"},
    refit="AUC",
    cv=5,
    verbose=0,
    n_jobs=-1,
)

In [98]:
# fit xgb_gs
xgb_gs.fit(tvshows_X_train, tvshows_y_train)

/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost

[16:21:38] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:21:38] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:21:38] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to '

/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost

[16:21:39] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:21:39] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:21:39] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to '

/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost

...

/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:22:39] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:22:39] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[16:22:42] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('xgb_transformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('onehot_titleType',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['titleType']),
                                                                        ('onehot_release_year',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['release_year']),
                                                                        ('bagofwords',
                                                                         CountVectorizer(min_df=0.02,
                                                      

In [99]:
print("XGBoost on TV SHOWS (BoW SUMMARY) best score =", xgb_gs.best_score_)
print("Test set AUC (BoW SUMMARY) = ", roc_auc_score(tvshows_y_test, xgb_gs.predict_proba(tvshows_X_test)[:,1]))

XGBoost on TV SHOWS (BoW SUMMARY) best score = 0.8208263657962045
Test set AUC (BoW SUMMARY) =  0.8582737845032926


## XGBoost - TF-IDF Summary

In [163]:
#import tfidf
from sklearn.feature_extraction.text import TfidfVectorizer

In [164]:
rankings_pipeline = Pipeline([
    #('imputer', SimpleImputer(strategy='mean')),
    ('logger', FunctionTransformer(np.log1p))
])

xgb_transformer = ColumnTransformer(
    [
        ("onehot_titleType",OneHotEncoder(handle_unknown='ignore'), ['titleType']),
        ("onehot_release_year",OneHotEncoder(handle_unknown='ignore'), ['release_year']),
        #("bagofwords", CountVectorizer(stop_words='english', min_df=0.02), 'summary'),             # One hot encoding
        ("tfidf", TfidfVectorizer(stop_words='english', min_df=0.02), 'summary'),                   # TF-IDF
        ("genres_count", CountVectorizer(min_df=0.05), 'genres'),
        #("sentence_transformer", embedder, 'summary'),                                             # hugging face - PCA
        #("imputer", SimpleImputer(strategy='mean'), ['averageRating', 'numVotes', 'runtimeMinutes']),
        ("rankings_pipeline", rankings_pipeline, ['director_rank', 'actor_rank', 'writer_rank']),
    ],
    remainder='passthrough')

In [165]:
# Recreate pipeline using XGBoost
xgb_pipe = Pipeline(
    steps=[
        ('xgb_transformer', xgb_transformer),
        #('normalizer', sklearn.preprocessing.Normalizer()),
        ('xgb', XGBClassifier())
    ]
)

xgb_gs = GridSearchCV(
    xgb_pipe,
    param_grid={
        'xgb__max_depth': [3, 5, 7],
        'xgb__learning_rate': [0.01, 0.1, 0.5],
        'xgb__n_estimators': [100, 200, 300]
    },
    scoring={"AUC": "roc_auc", "Accuracy": "accuracy"},
    refit="AUC",
    cv=5,
    verbose=0,
    n_jobs=-1,
)

In [166]:
# fit xgb_gs
xgb_gs.fit(tvshows_X_train, tvshows_y_train)

/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future ve

[17:04:03] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:04:03] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:04:03] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to '

/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost

[17:04:05] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:04:05] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:04:05] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to '

/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost

...

/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:05:18] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:05:18] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[17:05:21] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('xgb_transformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('onehot_titleType',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['titleType']),
                                                                        ('onehot_release_year',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['release_year']),
                                                                        ('tfidf',
                                                                         TfidfVectorizer(min_df=0.02,
                                                           

In [167]:
print("XGBoost on TV SHOWS (TFIDF SUMMARY) best score =", xgb_gs.best_score_)
print("Test set AUC (TFIDF SUMMARY) = ", roc_auc_score(tvshows_y_test, xgb_gs.predict_proba(tvshows_X_test)[:,1]))

XGBoost on TV SHOWS (TFIDF SUMMARY) best score = 0.8292691851267285
Test set AUC (TFIDF SUMMARY) =  0.8362757461118118


In [168]:
# save xgb_gs as pickle file
import pickle
pickle.dump(xgb_gs, open("../src/models/xgb_gs.pkl", "wb"))

## XGBoost - Summary Sentence Transformer (No PCA; All 389)

In [105]:
# append summary_embeddings_df as new columns to df
df_v2 = pd.concat([df, summary_embeddings_df], axis=1)


In [106]:
#Split movie and tv_show
movies_v2, tv_shows_v2 = df_v2[df_v2['is_tv_show']==False], df_v2[df_v2['is_tv_show']==True]

In [107]:
# remove tv show rows with season >= 24
# THERE ARE 2 TV SHOWS WITH SEASON = 24 AND SEASON = 2806. NEED TO REMOVE
tv_shows_v2 = tv_shows_v2[tv_shows_v2['season']<24]

# drop combined_title, release_date, lower_title, is_tv_show, tconst
tv_shows_v2 = tv_shows_v2.drop(['combined_title', 'release_date', 'lower_title', 'is_tv_show', 'tconst', 'summary'], axis=1)
tv_shows_v2

,release_year,is_top10,season,averageRating,numVotes,titleType,runtimeMinutes,genres,release_date_quarter,release_date_month,...,summary_embedding_374,summary_embedding_375,summary_embedding_376,summary_embedding_377,summary_embedding_378,summary_embedding_379,summary_embedding_380,summary_embedding_381,summary_embedding_382,summary_embedding_383
0,2020.0,0.0,1.0,6.8,4915.0,tvSeries,36.0,Comedy,2.0,4.0,...,0.073990,-0.024292,-0.000128,0.056980,-0.071850,0.032407,0.101245,0.041416,0.107043,-0.010314
1,2020.0,0.0,1.0,6.4,1099.0,tvSeries,NaN,Documentary,3.0,8.0,...,-0.009092,0.048180,0.053298,-0.018979,-0.034616,-0.023176,0.066810,-0.137848,0.092657,0.052799
3,2020.0,0.0,1.0,5.8,4179.0,tvSeries,NaN,Reality-TV,1.0,3.0,...,0.090168,0.066829,0.020300,0.105946,-0.012541,0.031277,0.135142,0.099034,-0.108421,-0.074679
6,2020.0,1.0,4.0,7.5,291371.0,tvSeries,60.0,"Drama,Mystery,Thriller",2.0,6.0,...,-0.035460,-0.031576,-0.047336,0.058432,-0.057747,-0.017877,0.047018,0.072994,0.011252,-0.030472
11,2020.0,1.0,4.0,7.3,25863.0,tvSeries,49.0,"Action,Drama,Sci-Fi",3.0,8.0,...,0.030561,0.022455,0.075101,0.093642,-0.020394,0.000410,0.079297,-0.060786,-0.101086,0.036998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2215,2021.0,1.0,1.0,5.7,1486.0,tvSeries,35.0,"Comedy,Drama,Family",1.0,3.0,...,0.006454,-0.006329,-0.031277,0.060810,0.032114,0.052678,0.021894,0.065260,0.007671,0.002937
2217,2021.0,0.0,1.0,5.7,1344.0,tvSeries,27.0,"Action,Comedy,Drama",2.0,4.0,...,0.007984,0.026756,-0.018391,0.017888,-0.068308,0.026351,-0.000278,0.032704,-0.022736,-0.035268
2219,2022.0,1.0,1.0,8.0,9215.0,tvMiniSeries,90.0,"Documentary,Music",1.0,2.0,...,0.114481,-0.020720,-0.023883,0.116682,-0.142948,-0.001545,0.030206,0.061836,-0.017303,0.028517
2220,2020.0,0.0,1.0,2.6,3361.0,tvSeries,30.0,"Documentary,Reality-TV",1.0,1.0,...,0.013962,0.076438,-0.014512,0.021898,0.000207,0.132097,0.042375,-0.010665,0.010991,-0.027117


In [108]:
# split into training and test
tvshows_v2_X_train, tvshows_v2_X_test, tvshows_v2_y_train, tvshows_v2_y_test = train_test_split(tv_shows_v2.drop('is_top10', axis=1), tv_shows_v2['is_top10'], test_size=0.2, random_state=42)

In [109]:
rankings_pipeline = Pipeline([
    #('imputer', SimpleImputer(strategy='mean')),
    ('logger', FunctionTransformer(np.log1p))
])

xgb_transformer = ColumnTransformer(
    [
        ("onehot_titleType",OneHotEncoder(handle_unknown='ignore'), ['titleType']),
        ("onehot_release_year",OneHotEncoder(handle_unknown='ignore'), ['release_year']),
        #("bagofwords", CountVectorizer(stop_words='english', min_df=0.02), 'summary'),             # One hot encoding
        #("tfidf", TfidfVectorizer(stop_words='english', min_df=0.02), 'summary'),                   # TF-IDF
        ("genres_count", CountVectorizer(min_df=0.05), 'genres'),
        #("sentence_transformer", embedder, 'summary'),                                             # hugging face - PCA
        #("imputer", SimpleImputer(strategy='mean'), ['averageRating', 'numVotes', 'runtimeMinutes']),
        ("rankings_pipeline", rankings_pipeline, ['director_rank', 'actor_rank', 'writer_rank']),
    ],
    remainder='passthrough')

In [110]:
# Recreate pipeline using XGBoost
xgb_pipe = Pipeline(
    steps=[
        ('xgb_transformer', xgb_transformer),
        #('normalizer', sklearn.preprocessing.Normalizer()),
        ('xgb', XGBClassifier())
    ]
)

xgb_gs = GridSearchCV(
    xgb_pipe,
    param_grid={
        'xgb__max_depth': [3, 5, 7],
        'xgb__learning_rate': [0.01, 0.1, 0.5],
        'xgb__n_estimators': [100, 200, 300]
    },
    scoring={"AUC": "roc_auc", "Accuracy": "accuracy"},
    refit="AUC",
    cv=5,
    verbose=0,
    n_jobs=-1,
)

In [111]:
# fit xgb_gs
xgb_gs.fit(tvshows_v2_X_train, tvshows_v2_y_train)

/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost

[16:23:55] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:23:55] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:23:55] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to '

/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost

[16:24:01] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:24:01] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:24:01] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to '

/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:28:57] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[16:29:03] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('xgb_transformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('onehot_titleType',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['titleType']),
                                                                        ('onehot_release_year',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['release_year']),
                                                                        ('genres_count',
                                                                         CountVectorizer(min_df=0.05),
                                                   

In [112]:
print("XGBoost on TV SHOWS (SENTENCE TRANSFORMER, NO PCA, SUMMARY) best score =", xgb_gs.best_score_)
print("Test set AUC (SENTENCE TRANSFORMER, NO PCA, SUMMARY) = ", roc_auc_score(tvshows_v2_y_test, xgb_gs.predict_proba(tvshows_v2_X_test)[:,1]))

XGBoost on TV SHOWS (SENTENCE TRANSFORMER, NO PCA, SUMMARY) best score = 0.7702127377531474
Test set AUC (SENTENCE TRANSFORMER, NO PCA, SUMMARY) =  0.7780042270531401


## XGBoost - PCA Sentence Transformer Summary (10 Components)

In [113]:
nComponents = 10
pca = PCA(n_components=nComponents)
summary_embeddings_pca_df = pca.fit_transform(summary_embeddings_df)

In [114]:
pd.DataFrame(summary_embeddings_pca_df)

,0,1,2,3,4,5,6,7,8,9
0,0.031476,0.247978,-0.029863,-0.204543,0.070628,0.079916,-0.149686,-0.246144,-0.011378,-0.054940
1,-0.227691,-0.098764,-0.137208,0.137831,0.124026,-0.023160,0.097528,-0.087570,0.059895,-0.066785
2,-0.117274,0.045485,0.121807,-0.121051,-0.024990,0.059883,0.096262,0.121531,-0.032504,-0.102865
3,-0.080938,-0.044293,0.098556,0.300531,0.111965,-0.041644,0.061355,-0.136750,0.058050,0.184262
4,-0.042110,0.031154,0.069575,-0.223254,0.303658,-0.039099,-0.172624,-0.104349,-0.022310,-0.091721
...,...,...,...,...,...,...,...,...,...,...
2221,-0.168701,-0.019844,-0.310131,-0.124630,-0.041231,-0.102957,-0.091003,-0.232536,-0.162666,0.014556
2222,-0.212914,-0.019794,-0.023129,0.198072,-0.000709,-0.137236,-0.025719,-0.038278,0.178822,0.059028
2223,-0.199555,0.103026,-0.122204,-0.103816,-0.051199,-0.113323,-0.103599,-0.319141,-0.134675,-0.003845
2224,0.254867,0.105872,-0.267038,0.275877,-0.063850,-0.138830,-0.019574,-0.011102,-0.038106,-0.047293


In [115]:
df.shape

(2221, 20)

In [116]:
# append summary_embeddings_df as new columns to df
df_v3 = pd.concat([df, pd.DataFrame(summary_embeddings_pca_df)], axis=1)


In [117]:
#Split movie and tv_show
movies_v3, tv_shows_v3 = df_v3[df_v3['is_tv_show']==False], df_v3[df_v3['is_tv_show']==True]

In [118]:
# remove tv show rows with season >= 24
# THERE ARE 2 TV SHOWS WITH SEASON = 24 AND SEASON = 2806. NEED TO REMOVE
tv_shows_v3 = tv_shows_v3[tv_shows_v3['season']<24]

# drop combined_title, release_date, lower_title, is_tv_show, tconst
tv_shows_v3 = tv_shows_v3.drop(['combined_title', 'release_date', 'lower_title', 'is_tv_show', 'tconst', 'summary'], axis=1)
tv_shows_v3

,release_year,is_top10,season,averageRating,numVotes,titleType,runtimeMinutes,genres,release_date_quarter,release_date_month,...,0,1,2,3,4,5,6,7,8,9
0,2020.0,0.0,1.0,6.8,4915.0,tvSeries,36.0,Comedy,2.0,4.0,...,0.031476,0.247978,-0.029863,-0.204543,0.070628,0.079916,-0.149686,-0.246144,-0.011378,-0.054940
1,2020.0,0.0,1.0,6.4,1099.0,tvSeries,NaN,Documentary,3.0,8.0,...,-0.227691,-0.098764,-0.137208,0.137831,0.124026,-0.023160,0.097528,-0.087570,0.059895,-0.066785
3,2020.0,0.0,1.0,5.8,4179.0,tvSeries,NaN,Reality-TV,1.0,3.0,...,-0.080938,-0.044293,0.098556,0.300531,0.111965,-0.041644,0.061355,-0.136750,0.058050,0.184262
6,2020.0,1.0,4.0,7.5,291371.0,tvSeries,60.0,"Drama,Mystery,Thriller",2.0,6.0,...,0.053909,0.076281,0.165015,-0.012370,-0.037178,-0.033755,-0.256471,0.022906,-0.017647,0.083480
11,2020.0,1.0,4.0,7.3,25863.0,tvSeries,49.0,"Action,Drama,Sci-Fi",3.0,8.0,...,0.055229,-0.087839,0.084608,0.232284,0.063037,-0.073815,0.084727,-0.220325,-0.183001,-0.105110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2215,2021.0,1.0,1.0,5.7,1486.0,tvSeries,35.0,"Comedy,Drama,Family",1.0,3.0,...,0.024053,0.287122,0.135127,-0.079178,-0.269147,-0.292347,0.041646,0.105765,0.055842,-0.188856
2217,2021.0,0.0,1.0,5.7,1344.0,tvSeries,27.0,"Action,Comedy,Drama",2.0,4.0,...,-0.037161,0.099416,0.167902,-0.069574,-0.249975,0.220239,-0.121435,0.073136,-0.053810,0.006001
2219,2022.0,1.0,1.0,8.0,9215.0,tvMiniSeries,90.0,"Documentary,Music",1.0,2.0,...,0.158586,-0.004224,0.179569,0.168729,-0.293055,-0.087068,-0.351368,-0.155067,-0.039701,0.094224
2220,2020.0,0.0,1.0,2.6,3361.0,tvSeries,30.0,"Documentary,Reality-TV",1.0,1.0,...,-0.333791,0.058498,-0.225210,0.106500,-0.170713,-0.083639,0.082494,0.070949,-0.004284,-0.073008


In [119]:
# split into training and test
tvshows_v3_X_train, tvshows_v3_X_test, tvshows_v3_y_train, tvshows_v3_y_test = train_test_split(tv_shows_v3.drop('is_top10', axis=1), tv_shows_v3['is_top10'], test_size=0.2, random_state=42)

In [120]:
rankings_pipeline = Pipeline([
    #('imputer', SimpleImputer(strategy='mean')),
    ('logger', FunctionTransformer(np.log1p))
])

xgb_transformer = ColumnTransformer(
    [
        ("onehot_titleType",OneHotEncoder(handle_unknown='ignore'), ['titleType']),
        ("onehot_release_year",OneHotEncoder(handle_unknown='ignore'), ['release_year']),
        #("bagofwords", CountVectorizer(stop_words='english', min_df=0.02), 'summary'),             # One hot encoding
        #("tfidf", TfidfVectorizer(stop_words='english', min_df=0.02), 'summary'),                   # TF-IDF
        ("genres_count", CountVectorizer(min_df=0.05), 'genres'),
        #("sentence_transformer", embedder, 'summary'),                                             # hugging face - PCA
        #("imputer", SimpleImputer(strategy='mean'), ['averageRating', 'numVotes', 'runtimeMinutes']),
        ("rankings_pipeline", rankings_pipeline, ['director_rank', 'actor_rank', 'writer_rank']),
    ],
    remainder='passthrough')

In [121]:
# Recreate pipeline using XGBoost
xgb_pipe = Pipeline(
    steps=[
        ('xgb_transformer', xgb_transformer),
        #('normalizer', sklearn.preprocessing.Normalizer()),
        ('xgb', XGBClassifier())
    ]
)

xgb_gs = GridSearchCV(
    xgb_pipe,
    param_grid={
        'xgb__max_depth': [3, 5, 7],
        'xgb__learning_rate': [0.01, 0.1, 0.5],
        'xgb__n_estimators': [100, 200, 300]
    },
    scoring={"AUC": "roc_auc", "Accuracy": "accuracy"},
    refit="AUC",
    cv=5,
    verbose=0,
    n_jobs=-1,
)

In [122]:
# fit xgb_gs
xgb_gs.fit(tvshows_v3_X_train, tvshows_v3_y_train)

/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/yenhann/opt

[16:29:09] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:29:09] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:29:09] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to '

/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/yenhann/opt

[16:29:10] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:29:10] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:29:10] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to '

/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/yenhann/opt

...

/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/yenhann/opt

[16:30:40] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[16:30:40] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[16:30:44] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('xgb_transformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('onehot_titleType',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['titleType']),
                                                                        ('onehot_release_year',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['release_year']),
                                                                        ('genres_count',
                                                                         CountVectorizer(min_df=0.05),
                                                   

In [123]:
print("XGBoost on TV SHOWS (SENTENCE TRANSFORMER, NO PCA, SUMMARY) best score =", xgb_gs.best_score_)
print("Test set AUC (SENTENCE TRANSFORMER, NO PCA, SUMMARY) = ", roc_auc_score(tvshows_v3_y_test, xgb_gs.predict_proba(tvshows_v3_X_test)[:,1]))

XGBoost on TV SHOWS (SENTENCE TRANSFORMER, NO PCA, SUMMARY) best score = 0.7731008184523811
Test set AUC (SENTENCE TRANSFORMER, NO PCA, SUMMARY) =  0.8596014492753623


## Best to Worst on Validation Set

1) TFIDF
2) No Summary
3) Bag of Words
4) Sentence Transformer

# Random Forest

## Random Forest - No Summary

In [138]:
# make sklearn pipeline with transform, normalization, and model (RF)
from sklearn.ensemble import RandomForestClassifier

rankings_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('logger', FunctionTransformer(np.log1p))
])

rf_transformer = ColumnTransformer(
    [
        ("onehot_titleType",OneHotEncoder(handle_unknown='ignore'), ['titleType']),
        ("onehot_release_year",OneHotEncoder(handle_unknown='ignore'), ['release_year']),
        #("bagofwords", CountVectorizer(stop_words='english', min_df=0.02), 'summary'),             # One hot encoding
        #("tfidf", TfidfVectorizer(stop_words='english', min_df=0.02), 'summary'),                   # TF-IDF
        ("genres_count", CountVectorizer(min_df=0.05), 'genres'),
        #("sentence_transformer", embedder, 'summary'),                                             # hugging face - PCA
        ("imputer", SimpleImputer(strategy='mean'), ['averageRating', 'numVotes', 'runtimeMinutes']),
        ("rankings_pipeline", rankings_pipeline, ['director_rank', 'actor_rank', 'writer_rank']),
    ],
    remainder='passthrough')

# make sklearn pipeline with transform, normalization, and model (RF)
pipeRF = Pipeline(
    steps=[
        ('rf_transformer', rf_transformer),                                                       # Transform dataset                
        #('normalizer', sklearn.preprocessing.Normalizer()),                                 # Normalise features
        #('standardizer', sklearn.preprocessing.StandardScaler(with_mean=False)),            # Rescale dataset
        ('randomforest', RandomForestClassifier())                             # Build Decision Tree model
    ]
)    


In [139]:
gsRF = GridSearchCV(
    pipeRF,
    param_grid={
        'randomforest__n_estimators': [100, 200, 300, 400, 500],
        'randomforest__max_depth': [5, 10, 15, 20],
        'randomforest__min_samples_leaf': [15, 20, 25, 30, 35, 40],
    },
    scoring={"AUC": "roc_auc", "Accuracy": "accuracy"},
    refit="AUC",
    cv=5,
    verbose=0,
    n_jobs=-1,
)

In [140]:
gsRF.fit(tvshows_X_train.drop(columns=['summary']), tvshows_y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('rf_transformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('onehot_titleType',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['titleType']),
                                                                        ('onehot_release_year',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['release_year']),
                                                                        ('genres_count',
                                                                         CountVectorizer(min_df=0.05),
                                                    

In [142]:
print("Random Forest on TV SHOWS (NO SUMMARY) best score =", gsRF.best_score_)
print("Test set AUC (NO SUMMARY) = ", roc_auc_score(tvshows_y_test, gsRF.predict_proba(tvshows_X_test)[:,1]))

Random Forest on TV SHOWS (NO SUMMARY) best score = 0.7669177641138243
Test set AUC (NO SUMMARY) =  0.7715426649852879


## Random Forest - Bag of Words Summary

In [143]:
# make sklearn pipeline with transform, normalization, and model (RF)
from sklearn.ensemble import RandomForestClassifier

rankings_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('logger', FunctionTransformer(np.log1p))
])

rf_transformer = ColumnTransformer(
    [
        ("onehot_titleType",OneHotEncoder(handle_unknown='ignore'), ['titleType']),
        ("onehot_release_year",OneHotEncoder(handle_unknown='ignore'), ['release_year']),
        ("bagofwords", CountVectorizer(stop_words='english', min_df=0.02), 'summary'),             # One hot encoding
        #("tfidf", TfidfVectorizer(stop_words='english', min_df=0.02), 'summary'),                   # TF-IDF
        ("genres_count", CountVectorizer(min_df=0.05), 'genres'),
        #("sentence_transformer", embedder, 'summary'),                                             # hugging face - PCA
        ("imputer", SimpleImputer(strategy='mean'), ['averageRating', 'numVotes', 'runtimeMinutes']),
        ("rankings_pipeline", rankings_pipeline, ['director_rank', 'actor_rank', 'writer_rank']),
    ],
    remainder='passthrough')

# make sklearn pipeline with transform, normalization, and model (RF)
pipeRF = Pipeline(
    steps=[
        ('rf_transformer', rf_transformer),                                                       # Transform dataset                
        #('normalizer', sklearn.preprocessing.Normalizer()),                                 # Normalise features
        #('standardizer', sklearn.preprocessing.StandardScaler(with_mean=False)),            # Rescale dataset
        ('randomforest', RandomForestClassifier())                             # Build Decision Tree model
    ]
)    


In [144]:
gsRF = GridSearchCV(
    pipeRF,
    param_grid={
        'randomforest__n_estimators': [100, 200, 300, 400, 500],
        'randomforest__max_depth': [5, 10, 15, 20],
        'randomforest__min_samples_leaf': [15, 20, 25, 30, 35, 40],
    },
    scoring={"AUC": "roc_auc", "Accuracy": "accuracy"},
    refit="AUC",
    cv=5,
    verbose=0,
    n_jobs=-1,
)

In [145]:
gsRF.fit(tvshows_X_train, tvshows_y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('rf_transformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('onehot_titleType',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['titleType']),
                                                                        ('onehot_release_year',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['release_year']),
                                                                        ('bagofwords',
                                                                         CountVectorizer(min_df=0.02,
                                                       

In [146]:
print("Random Forest on TV SHOWS (BoW SUMMARY) best score =", gsRF.best_score_)
print("Test set AUC (BoW SUMMARY) = ", roc_auc_score(tvshows_y_test, gsRF.predict_proba(tvshows_X_test)[:,1]))

Random Forest on TV SHOWS (NO SUMMARY) best score = 0.7601346193961745
Test set AUC (NO SUMMARY) =  0.7608939330250805


## Random Forest - TFIDF Summary

In [147]:
# make sklearn pipeline with transform, normalization, and model (RF)
from sklearn.ensemble import RandomForestClassifier

rankings_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('logger', FunctionTransformer(np.log1p))
])

rf_transformer = ColumnTransformer(
    [
        ("onehot_titleType",OneHotEncoder(handle_unknown='ignore'), ['titleType']),
        ("onehot_release_year",OneHotEncoder(handle_unknown='ignore'), ['release_year']),
        #("bagofwords", CountVectorizer(stop_words='english', min_df=0.02), 'summary'),             # One hot encoding
        ("tfidf", TfidfVectorizer(stop_words='english', min_df=0.02), 'summary'),                   # TF-IDF
        ("genres_count", CountVectorizer(min_df=0.05), 'genres'),
        #("sentence_transformer", embedder, 'summary'),                                             # hugging face - PCA
        ("imputer", SimpleImputer(strategy='mean'), ['averageRating', 'numVotes', 'runtimeMinutes']),
        ("rankings_pipeline", rankings_pipeline, ['director_rank', 'actor_rank', 'writer_rank']),
    ],
    remainder='passthrough')

# make sklearn pipeline with transform, normalization, and model (RF)
pipeRF = Pipeline(
    steps=[
        ('rf_transformer', rf_transformer),                                                       # Transform dataset                
        #('normalizer', sklearn.preprocessing.Normalizer()),                                 # Normalise features
        #('standardizer', sklearn.preprocessing.StandardScaler(with_mean=False)),            # Rescale dataset
        ('randomforest', RandomForestClassifier())                             # Build Decision Tree model
    ]
)    


In [148]:
gsRF = GridSearchCV(
    pipeRF,
    param_grid={
        'randomforest__n_estimators': [100, 200, 300, 400, 500],
        'randomforest__max_depth': [5, 10, 15, 20],
        'randomforest__min_samples_leaf': [15, 20, 25, 30, 35, 40],
    },
    scoring={"AUC": "roc_auc", "Accuracy": "accuracy"},
    refit="AUC",
    cv=5,
    verbose=0,
    n_jobs=-1,
)

In [149]:
gsRF.fit(tvshows_X_train, tvshows_y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('rf_transformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('onehot_titleType',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['titleType']),
                                                                        ('onehot_release_year',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['release_year']),
                                                                        ('tfidf',
                                                                         TfidfVectorizer(min_df=0.02,
                                                            

In [150]:
print("Random Forest on TV SHOWS (TFIDF SUMMARY) best score =", gsRF.best_score_)
print("Test set AUC (TFIDF SUMMARY) = ", roc_auc_score(tvshows_y_test, gsRF.predict_proba(tvshows_X_test)[:,1]))

Random Forest on TV SHOWS (NO SUMMARY) best score = 0.7582555268074066
Test set AUC (NO SUMMARY) =  0.7592125542945216


## Random Forest - Sentence Transformer All Embeddings Summary

In [151]:
# make sklearn pipeline with transform, normalization, and model (RF)
from sklearn.ensemble import RandomForestClassifier

rankings_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('logger', FunctionTransformer(np.log1p))
])

rf_transformer = ColumnTransformer(
    [
        ("onehot_titleType",OneHotEncoder(handle_unknown='ignore'), ['titleType']),
        ("onehot_release_year",OneHotEncoder(handle_unknown='ignore'), ['release_year']),
        #("bagofwords", CountVectorizer(stop_words='english', min_df=0.02), 'summary'),             # One hot encoding
        #("tfidf", TfidfVectorizer(stop_words='english', min_df=0.02), 'summary'),                   # TF-IDF
        ("genres_count", CountVectorizer(min_df=0.05), 'genres'),
        #("sentence_transformer", embedder, 'summary'),                                             # hugging face - PCA
        ("imputer", SimpleImputer(strategy='mean'), ['averageRating', 'numVotes', 'runtimeMinutes']),
        ("rankings_pipeline", rankings_pipeline, ['director_rank', 'actor_rank', 'writer_rank']),
    ],
    remainder='passthrough')

# make sklearn pipeline with transform, normalization, and model (RF)
pipeRF = Pipeline(
    steps=[
        ('rf_transformer', rf_transformer),                                                       # Transform dataset                
        #('normalizer', sklearn.preprocessing.Normalizer()),                                 # Normalise features
        #('standardizer', sklearn.preprocessing.StandardScaler(with_mean=False)),            # Rescale dataset
        ('randomforest', RandomForestClassifier())                             # Build Decision Tree model
    ]
)    


In [152]:
gsRF = GridSearchCV(
    pipeRF,
    param_grid={
        'randomforest__n_estimators': [100, 200, 300, 400, 500],
        'randomforest__max_depth': [5, 10, 15, 20],
        'randomforest__min_samples_leaf': [15, 20, 25, 30, 35, 40],
    },
    scoring={"AUC": "roc_auc", "Accuracy": "accuracy"},
    refit="AUC",
    cv=5,
    verbose=0,
    n_jobs=-1,
)

In [153]:
gsRF.fit(tvshows_v2_X_train, tvshows_v2_y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('rf_transformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('onehot_titleType',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['titleType']),
                                                                        ('onehot_release_year',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['release_year']),
                                                                        ('genres_count',
                                                                         CountVectorizer(min_df=0.05),
                                                    

In [155]:
print("Random Forest on TV SHOWS (NO SUMMARY) best score =", gsRF.best_score_)
print("Test set AUC (NO SUMMARY) = ", roc_auc_score(tvshows_v2_y_test, gsRF.predict_proba(tvshows_v2_X_test)[:,1]))

Random Forest on TV SHOWS (NO SUMMARY) best score = 0.7440685293513957
Test set AUC (NO SUMMARY) =  0.7865338164251209


## Random Forest - Sentence Transformer Embeddings PCA (10 Components)

In [159]:
# make sklearn pipeline with transform, normalization, and model (RF)
from sklearn.ensemble import RandomForestClassifier

rankings_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('logger', FunctionTransformer(np.log1p))
])

rf_transformer = ColumnTransformer(
    [
        ("onehot_titleType",OneHotEncoder(handle_unknown='ignore'), ['titleType']),
        ("onehot_release_year",OneHotEncoder(handle_unknown='ignore'), ['release_year']),
        #("bagofwords", CountVectorizer(stop_words='english', min_df=0.02), 'summary'),             # One hot encoding
        #("tfidf", TfidfVectorizer(stop_words='english', min_df=0.02), 'summary'),                   # TF-IDF
        ("genres_count", CountVectorizer(min_df=0.05), 'genres'),
        #("sentence_transformer", embedder, 'summary'),                                             # hugging face - PCA
        ("imputer", SimpleImputer(strategy='mean'), ['averageRating', 'numVotes', 'runtimeMinutes']),
        ("rankings_pipeline", rankings_pipeline, ['director_rank', 'actor_rank', 'writer_rank']),
    ],
    remainder='passthrough')

# make sklearn pipeline with transform, normalization, and model (RF)
pipeRF = Pipeline(
    steps=[
        ('rf_transformer', rf_transformer),                                                       # Transform dataset                
        #('normalizer', sklearn.preprocessing.Normalizer()),                                 # Normalise features
        #('standardizer', sklearn.preprocessing.StandardScaler(with_mean=False)),            # Rescale dataset
        ('randomforest', RandomForestClassifier())                             # Build Decision Tree model
    ]
)    


In [160]:
gsRF = GridSearchCV(
    pipeRF,
    param_grid={
        'randomforest__n_estimators': [100, 200, 300, 400, 500],
        'randomforest__max_depth': [5, 10, 15, 20],
        'randomforest__min_samples_leaf': [15, 20, 25, 30, 35, 40],
    },
    scoring={"AUC": "roc_auc", "Accuracy": "accuracy"},
    refit="AUC",
    cv=5,
    verbose=0,
    n_jobs=-1,
)

In [161]:
gsRF.fit(tvshows_v3_X_train, tvshows_v3_y_train)

/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/Users/yenhann/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be rais

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('rf_transformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('onehot_titleType',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['titleType']),
                                                                        ('onehot_release_year',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['release_year']),
                                                                        ('genres_count',
                                                                         CountVectorizer(min_df=0.05),
                                                    

In [162]:
print("Random Forest on TV SHOWS (NO SUMMARY) best score =", gsRF.best_score_)
print("Test set AUC (NO SUMMARY) = ", roc_auc_score(tvshows_v3_y_test, gsRF.predict_proba(tvshows_v3_X_test)[:,1]))

Random Forest on TV SHOWS (NO SUMMARY) best score = 0.748368247981664
Test set AUC (NO SUMMARY) =  0.7909873188405797


## Best to Worst on Validation Set

1) No Summary
2) Bag of Words
3) TFIDF
4) Sentence Transformer